# 从零开始部署领域大模型应用——以DST-GPT为例

![Bg Image](../assets/BG.png)

In [ ]:
# INSTALL PACKAGES
# using conda env | python=3.8.18
%pip install langchain
%pip install langchain-openai
%pip install python-dotenv

# [Quickstart | 🦜️🔗 Langchain](https://python.langchain.com/docs/get_started/quickstart)

> 由于我们使用的是OpenAI的服务，使用此应用时可能需要用到魔法  
> 但如果你的OpenAI API key是中转类型的，可能不需要

## 1. 配置OpenAI API key

In [1]:
# Data Privacy
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Set up your API key from OpenAI
api_key = os.getenv('OPENAI_API_KEY')
base_url=os.getenv('OPENAI_BASE_URL')

# Print
print(api_key)
print(base_url)

sk-zYQdjPJq1UgnMUTYEc8c38997bCb4c089768D16b2965FcD5
https://api.kwwai.top/v1


## 2. 创建一个基本LLM应用

然后，我们可以初始化模型：

In [4]:
# Initialize the model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key, # alias:api_key
    base_url="", # If you use proxy api key, set base_url as needed 
    model="gpt-3.5-turbo-0125", 
    temperature=0.7, #default
    )

一旦您安装并初始化了您选择的LLM，我们就可以尝试使用它了！让我们问它 '你调用的是哪个基模型'。
> 大模型具有随机性的特点，完全相同的输入可以得到不同（但相近）的回答

In [5]:
for i in range(5):
    response = llm.invoke("你用的是哪个底座模型")
    print(f"Response {i+1}: {response.content}")

Response 1: 很抱歉，我不是一个实际存在的机器人，我是一个基于人工智能的语言模型，没有具体的底座模型。我是在云端服务器上运行的程序，通过处理输入的文本来生成回复。您有什么问题可以问我，我会尽力帮助解答。
Response 2: 我是基于OpenAI的GPT-3模型进行训练的。
Response 3: 我是基于GPT-3模型进行训练的。
Response 4: 我是一个数字助手，没有具体的底座模型。我是基于自然语言处理技术和机器学习算法构建的人工智能系统，可以回答您的问题和提供帮助。您有什么问题可以问我。
Response 5: 作为一个语言模型AI助手，我没有实际的物理底座模型，我只是一个程序在服务器上运行，通过文字进行交流。你有任何问题需要帮助吗？


## 3. 使用向量数据库外挂知识增强LLM在特定领域的能力——以Don't Starve Together(DST)为例

![Sup1](../assets/DST1.png)

我们向gpt-3.5-turbo-0125提一个关于DST这款游戏的问题，即“Don't Starve Together这款游戏截止2023年12月，在Steam平台上共有多少评论？”。由于gpt-3.5-turbo-0125的训练语料数据仅截止到2021年过，显然其目前无法回答这个问题。此外，此问题过于领域化，基座大模型训练语料基本不可能包含相关信息，我们这里通过向量数据库外挂知识的方式，增加LLM在特地领域的问答能力。

In [5]:
response=llm.invoke("Don't Starve Together这款游戏截止2023年12月，在Steam平台上共有多少评论？")
print(response.content)

截至2023年12月，在Steam平台上，Don't Starve Together这款游戏共有超过二十万条评论。


上述回答为大模型幻觉（Illusion）

为了正确回答此问题，我们需要为LLM添加额外知识。我们可以通过`Retrieval`来做到这一点。当您有太多数据无法直接传递时，`Retrieval`很有用。然后，您可以使用`Retriever`仅获取最相关的片段并将其传递。
在这个过程中，我们将从 `Retriever` 中查找相关文档，然后将它们传递到`Prompt`中。在本例中，我们将填充一个`vectorstore`并将其用作`Retriever`。有关 `vectorstore` 的更多信息，请参阅[此文档](https://python.langchain.com/docs/modules/data_connection/vectorstores)。

首先，我们需要加载要索引的数据。这里我们的数据来源于网页信息，为此，我们将使用 WebBaseLoader。这需要安装 BeautifulSoup：

In [6]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


之后，我们可以导入并使用 WebBaseLoader。

In [7]:
from langchain_community.document_loaders import WebBaseLoader
dst_wiki_mainpage_url="https://dontstarve.fandom.com/wiki/Don%27t_Starve_Together#Return_of_Them"
loader = WebBaseLoader(dst_wiki_mainpage_url)

docs = loader.load()



In [8]:
print(docs[0].page_content[100:180])







Don't Starve Wiki





 Explore

 




 Main Page




 Discuss




All Pag


清洗数据，去掉"\n"和"\t"

In [9]:
# Get the text of the first Document instance
original_text = docs[0].page_content

# Use the replace method to remove newline and tab characters
cleaned_text = original_text.replace("\n", "").replace("\t", "")

# Update the text attribute of the Document instance
docs[0].page_content = cleaned_text

# Print the length of the cleaned text and the cleaned text itself for confirmation
print("文档字符串长度为：", len(docs[0].page_content))
#print(docs[0].page_content)

文档字符串长度为： 29544


截取这个问题的回答相关信息源，打印。

In [10]:
content = docs[0].page_content
start_index = content.find("As of 2023-12-05 on Steam, DST had")
end_index = content.find("new development takes place almost exclusively in DST.")
relevant_content = content[start_index:end_index]
print(relevant_content)

As of 2023-12-05 on Steam, DST had 294K reviews, DS 88K, Shipwrecked 3K, RoG 1.5K, and Hamlet 1.3K, indicating that DST is at least 3x as popular as DS was.  The other DLC's are relatively minor players.  Each version has its own charm, but 


接下来，我们需要将其索引到`vectorstore`中。这需要一些组件，即`embedding`和`vectorstore`。
对于`embedding`，我们再次使用OpenAI的示例。

确保您安装了“langchain_openai”包并设置了适当的环境变量（这些变量与 所需的环境变量相同LLM）。

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model= "text-embedding-ada-002",#default
    openai_api_key=api_key, # alias:api_key
    base_url=base_url, # If you use proxy api key, set base_url as needed 
    chunk_size=1000,#default
)

现在，我们可以使用这个`embedding`将文档摄取到`vectorstore`中。为了简单起见，我们将使用一个简单的本地`vectorstore`: `Chroma`

首先，我们需要为此安装所需的软件包：

In [ ]:
%pip install chromadb

然后我们可以构建我们的索引：

In [13]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

vector = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings,
    )

In [14]:
print(len(vector.get()['ids']))
print(type(vector))


1
<class 'langchain_community.vectorstores.chroma.Chroma'>


现在，我们已经在`vectorstore`中索引了这些数据，我们将创建一个`retrieval chain`。该`chain`将接受传入的问题，查找相关文档，然后将这些文档与原始问题一起传递到一个LLM中，并要求它回答原始问题。

首先，让我们设置一个`chain`，它接受一个问题和检索到的文档并生成一个答案。

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(
    llm=llm, 
    prompt=prompt,
    )

我们现在可以调用这个chain了。

In [16]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "DST这款游戏截止2023年12月，在Steam平台上共有多少评论？"})
print(response["answer"])

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


根据提供的内容，DST在Steam平台上截止2023年12月共有294K条评论。


上述回答完全正确 参照[Wiki Fandom](https://dontstarve.fandom.com/wiki/Don%27t_Starve_Together#Return_of_Them)  
![DST2.png](../assets/DST2.png)

## 4. 使用异步调用llm

参考[langchain官方文档](https://python.langchain.com/docs)

In [22]:
response=await llm.ainvoke("你用的什么基模型")
print(response.content)
print(type(response.content))

我是一个基于大型神经网络的AI助手，我的模型是由开发团队使用自然语言处理和深度学习技术搭建而成的。我没有一个特定的基模型，而是基于多种模型和算法的综合使用。
<class 'str'>


## 5.使用服务器存储向量数据库（Chroma） 

load json data `sample_data.json`

In [3]:
import json

with open('../data/sample_data.json', 'r') as file:
    sample_data = json.load(file)

In [4]:
print("type of variable:",type(sample_data))
print("length of JSON file:",len(sample_data))

type of variable: <class 'list'>
length of JSON file: 5


In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
            model="text-embedding-ada-002",  # default
            openai_api_key=api_key,  # alias:api_key
            base_url=base_url,  # If you use proxy api key, set base_url as needed
            chunk_size=1000,  # default
        )

In [8]:
# create the chroma client
import uuid

import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(settings=Settings(allow_reset=True))
client.reset()  # resets the database
collection = client.create_collection("my_collection")
stored_vectors=None

for i, data in enumerate(sample_data):
    print(f"Processing {i} Item!")
    # Splitting text into 1500-character chunks with 100-character overlap
    chunk_size = 1500
    overlap = 100
    chunks = [
        data["text"][i : i + chunk_size]
        for i in range(0, len(data["text"]), chunk_size - overlap)
    ]
    metadata = [{k: v for k, v in data.items() if k != "text"}]
    
    if i == 0:
        stored_vectors = Chroma.from_texts(
            texts=chunks,
            metadatas=metadata,
            embedding=embeddings,
            client=client,
            collection_name="my_collection",
        )
    else:
        stored_vectors.add_texts(
            texts=chunks,
            metadatas=metadata,
        )


# tell LangChain to use our client and collection name
db4 = Chroma(
    client=client,
    collection_name="my_collection",
    embedding_function=embeddings,
)
query = "Tell about hunger"
docs = db4.similarity_search(query)
print(docs[0].page_content)

ValueError: Could not connect to a Chroma server. Are you sure it is running?